## 主要參考資料為
* https://machinelearningmastery.com/encoder-decoder-attention-sequence-to-sequence-prediction-keras/
* https://medium.com/datalogue/attention-in-keras-1892773a4f22

### Without Attention

In [ ]:
from random import randint
from numpy import array
from numpy import argmax
from numpy import array_equal
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import RepeatVector

# generate a sequence of random integers
def generate_sequence(length, n_unique):
	return [randint(0, n_unique-1) for _ in range(length)]

# one hot encode sequence
def one_hot_encode(sequence, n_unique):
	encoding = list()
	for value in sequence:
		vector = [0 for _ in range(n_unique)]
		vector[value] = 1
		encoding.append(vector)
	return array(encoding)

# decode a one hot encoded string
def one_hot_decode(encoded_seq):
	return [argmax(vector) for vector in encoded_seq]

# prepare data for the LSTM
def get_pair(n_in, n_out, cardinality):
	# generate random sequence
	sequence_in = generate_sequence(n_in, cardinality)
	sequence_out = sequence_in[:n_out] + [0 for _ in range(n_in-n_out)]
	# one hot encode
	X = one_hot_encode(sequence_in, cardinality)
	y = one_hot_encode(sequence_out, cardinality)
	# reshape as 3D
	X = X.reshape((1, X.shape[0], X.shape[1]))
	y = y.reshape((1, y.shape[0], y.shape[1]))
	return X,y

# configure problem
n_features = 50
n_timesteps_in = 5
n_timesteps_out = 2
# define model
model = Sequential()
model.add(LSTM(150, input_shape=(n_timesteps_in, n_features)))
model.add(RepeatVector(n_timesteps_in))
model.add(LSTM(150, return_sequences=True))
model.add(TimeDistributed(Dense(n_features, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])


In [2]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 150)               120600    
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 5, 150)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 5, 150)            180600    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 5, 50)             7550      
Total params: 308,750
Trainable params: 308,750
Non-trainable params: 0
_________________________________________________________________


In [3]:
# train LSTM
for epoch in range(5000):
	# generate new random sequence
	X,y = get_pair(n_timesteps_in, n_timesteps_out, n_features)
	# fit model for one epoch on this sequence
	model.fit(X, y, epochs=1, verbose=2)
# evaluate LSTM
total, correct = 100, 0
for _ in range(total):
	X,y = get_pair(n_timesteps_in, n_timesteps_out, n_features)
	yhat = model.predict(X, verbose=0)
	if array_equal(one_hot_decode(y[0]), one_hot_decode(yhat[0])):
		correct += 1
print('Accuracy: %.2f%%' % (float(correct)/float(total)*100.0))
# spot check some examples
for _ in range(10):
	X,y = get_pair(n_timesteps_in, n_timesteps_out, n_features)
	yhat = model.predict(X, verbose=0)
	print('Expected:', one_hot_decode(y[0]), 'Predicted', one_hot_decode(yhat[0]))

Epoch 1/1
 - 0s - loss: 0.7122 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.1110 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3977 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7012 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7233 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.0499 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4558 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.6043 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5700 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5560 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7610 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4596 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3140 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.6904 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.0032 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.0270 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5730 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5058 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.6716 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.6795 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7138 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.6576 - acc: 0.8000
Epoch 1/1


 - 0s - loss: 0.4672 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4219 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.8589 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.6813 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7794 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3249 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3130 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.4028 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.1304 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.7632 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7960 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7129 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5375 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.0159 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.6128 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 0.6935 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4076 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5456 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4609 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3602 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.1692 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.9425 - acc: 0.8000
Epoch 1/1
 - 0s - lo

Epoch 1/1
 - 0s - loss: 0.1285 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.9483 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4974 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.2484 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7012 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.8010 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.6894 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4695 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7814 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4496 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3300 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3161 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.8277 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3158 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0935 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.9225 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5616 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4843 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.6665 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.8001 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4075 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3269 - acc: 0.8000
Epoch 1/1


Epoch 1/1
 - 0s - loss: 0.5883 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7943 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4297 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3308 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.6096 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.2974 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.7383 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.2261 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.5618 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4593 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4648 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3015 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4121 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.5106 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3652 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.2632 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.7765 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.0375 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.3638 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.2907 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.8758 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7508 - acc: 0.8000
Epoch 1/1


 - 0s - loss: 0.1473 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.2220 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.3095 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.7644 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4580 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5693 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.1738 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.5002 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.5321 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.2225 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.6977 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4463 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.8750 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7940 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.9691 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.2910 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0581 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.3758 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4418 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.2888 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.2165 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.4769 - acc: 0.8000
Epoch 1/1
 - 0s - lo

...

 - 0s - loss: 0.0654 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.3576 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4927 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.2102 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.8879 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.2939 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4888 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.6275 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.0570 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.4428 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.2115 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.1029 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.2568 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.0520 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.2788 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.5584 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5805 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3989 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.1363 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.2396 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.2712 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0215 - acc: 1.0000
Epoch 1/1
 - 0s - lo

Epoch 1/1
 - 0s - loss: 0.3019 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.5663 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.2260 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0883 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.4297 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3376 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0783 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0815 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.3756 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.1725 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.4980 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3420 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4534 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4411 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.2113 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.2190 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.1642 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.5827 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3505 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.2662 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.1993 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0496 - acc: 1.0000
Epoch 1/1


 - 0s - loss: 0.0773 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.1039 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.6055 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.8961 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.2200 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4171 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.1348 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0728 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.1754 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.4976 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.0628 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.5335 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.2453 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.5043 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.6622 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5443 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.0958 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.1665 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.2386 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.1361 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.3639 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.2077 - acc: 1.0000
Epoch 1/1
 - 0s - lo

Epoch 1/1
 - 0s - loss: 0.1674 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.2452 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.3020 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.6164 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3182 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.2657 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3804 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.1829 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.1592 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.3024 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.1079 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.1097 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.7618 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.2136 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.2273 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.3785 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5036 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.0420 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.7291 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.2743 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.1598 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.2477 - acc: 1.0000
Epoch 1/1


 - 0s - loss: 0.0631 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0512 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.2029 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.2600 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3749 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.0254 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.1004 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.3420 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.1127 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.2197 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.4830 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.2574 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.6997 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.1116 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.4485 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.1057 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.3137 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.0949 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.4578 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.2080 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.1785 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0621 - acc: 1.0000
Epoch 1/1
 - 0s - lo

### With Attention